# Quickstart to integrate Recommenders in AzureML Designer

This notebook shows how to integrate any algorithm in Recommenders library into AzureML Designer. 

[AzureML Designer](https://docs.microsoft.com/en-us/azure/machine-learning/concept-designer) lets you visually connect datasets and modules on an interactive canvas to create machine learning models. 

![img](https://recodatasets.blob.core.windows.net/images/designer-drag-and-drop.gif)

One of the features of AzureML Designer is that it is possible for developers to integrate any python library to make it available as a module. In this notebook are are going to show how to integrate [SAR](sar_movielens.ipynb) and several other modules in Designer


## Installation

The first step is to install AzureML Designer SDK. Assuming that you have installed the Recommenders environment `reco_base` as explained in the [SETUP.md](../../SETUP.md), you need to install:
```bash
conda activate reco_base
pip install keyring artifacts-keyring
pip install azureml-designer-tools==0.1.10.post8799295 --extra-index-url=https://msdata.pkgs.visualstudio.com/_packaging/azureml-modules%40Local/pypi/simple/ 
```

## Module implementation

The scenario that we are going to reproduce in Designer, as a reference example, is the content of the [SAR quickstart notebook](sar_movielens.ipynb). In it, we load a dataset, split it into train and test sets, train SAR algorithm, predict using the test set and compute several ranking metrics (precision at k, recall at k, MAP and nDCG).

For the pipeline that we want to create in Designer, we need to build the following modules:

- Stratified splitter
- SAR training
- SAR prediction
- Precision at k
- Recall at k
- MAP
- nDCG

The python code is defined with a python entry and a yaml file. All the python entries and yaml files for this pipeline can be found in [reco_utils/azureml/azureml_designer_modules](../../reco_utils/azureml/azureml_designer_modules).


### Define python entry

To illustrate how a python entry is defined we are going to explain the [precision at k entry](../../reco_utils/azureml/azureml_designer_modules/precision_at_k_entry.py). A simplified version of the code is shown next:

```python
# Dependencies
from azureml.studio.core.data_frame_schema import DataFrameSchema
from azureml.studio.core.io.data_frame_directory import (
    load_data_frame_from_directory,
    save_data_frame_to_directory,
)
from reco_utils.evaluation.python_evaluation import precision_at_k

# First, the input variables of precision_at_k are defined as argparse arguments
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--rating-true", help="True DataFrame.")
    parser.add_argument("--rating-pred", help="Predicted DataFrame.")
    parser.add_argument(
        "--col-user", type=str, help="A string parameter with column name for user."
    )
    # ... more arguments
    args, _ = parser.parse_known_args()

    # This module has two main inputs from the canvas, the true and predicted labels
    # they are loaded into the runtime as a pandas DataFrame
    rating_true = load_data_frame_from_directory(args.rating_true).data
    rating_pred = load_data_frame_from_directory(args.rating_pred).data

    # The python function is instantiated and the computation is performed
    eval_precision = precision_at_k(rating_true, rating_pred)
    
    # To output the result to Designer, we write it as a DataFrame
    score_result = pd.DataFrame({"precision_at_k": [eval_precision]})
    save_data_frame_to_directory(
        args.score_result,
        score_result,
        schema=DataFrameSchema.data_frame_to_dict(score_result),
    )
```


### Define yaml entry

Once we have the python entry, we need to create the yaml file that will interact with Designer, [precision_at_k.yaml](../../reco_utils/azureml/azureml_designer_modules/precision_at_k.yaml).

```yaml
name: Precision at K
id: efd1af54-0d31-42e1-b3d5-ce3b7c538705
version: 0.0.1
category: Recommenders/Metrics
description: "Precision at K metric from Recommenders repo: https://github.com/Microsoft/Recommenders."
inputs:
- name: Rating true
  type: DataFrameDirectory
  description: True DataFrame.
  port: true
- name: Rating pred
  type: DataFrameDirectory
  description: Predicted DataFrame.
  port: true
- name: User column
  type: String
  default: UserId
  description: Column name of user IDs.
outputs:
- name: Score
  type: DataFrameDirectory
  description: Precision at k (min=0, max=1).
  port: true
implementation:
  container:
    conda: sar_conda.yaml
    entry: azureml-designer-modules/entries/precision_at_k_entry.py
    args:
    - --rating-true
    - inputPath: Rating true
    - --rating-pred
    - inputPath: Rating pred
    - --col-user
    - --score-result
    - outputPath: Score
```

In the yaml file we can see a number of sections. The heading defines attributes like name, version or description. In the section inputs, all inputs are defined. The two main dataframes have ports, which can be connected to other modules. The inputs without port appear in a canvas menu. The output is defined as a DataFrame as well. The last section, implementation, defines the conda environment, the associated python entry and the arguments to the python file.

Once the full pipeline is designer, it should look like this:
![img](https://recodatasets.blob.core.windows.net/images/azureml_designer_sar_precisionatk.png)


## Module Registration

Once the code is implemented, we need to register it as an AzureML Designer custom module. The registration can be performed following these simple steps:

In [1]:
from azureml.core import Workspace
from azureml.studio.tools.module_repo import ModuleRepo

In [2]:
# Enter your AzureML subscription details
workspace_name = "XXXXXXX"
resource_group_name = "XXXXXXX"
subscription_id = "XXXXXXXX-XXXX-XXXX-XXXX-XXXXXXXXXXXX"
subscription_name = "XXXXXXX"

workspace_name = "cat-demo-3-euap"
resource_group_name = "cat-demo-rg"
subscription_id = "d128f140-94e6-4175-87a7-954b9d27db16"
subscription_name = "AML V1 Personal 1"

In [ ]:
ws = Workspace.get(name=workspace_name, 
                   resource_group=resource_group_name, 
                   subscription_id=subscription_id)

In [ ]:
repo_url = "https://github.com/microsoft/recommenders/tree/master"
repo_url = "https://github.com/microsoft/recommenders/tree/designer"

ModuleRepo.register(workspace=ws, module_source=repo_url)